In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [77]:
from sklearn.linear_model import LogisticRegression

In [78]:
df = pd.read_csv('./datasets/car.data', header=None)
df.shape

(1728, 7)

In [79]:
df.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [80]:
df.columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df.columns

Index(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class'], dtype='object')

In [81]:
df.isna().sum().sum()

0

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
buying      1728 non-null object
maint       1728 non-null object
doors       1728 non-null object
persons     1728 non-null object
lug_boot    1728 non-null object
safety      1728 non-null object
class       1728 non-null object
dtypes: object(7)
memory usage: 94.6+ KB


In [83]:
x = df.drop(columns='class')
y = df.iloc[:, -1]
type(x), type(y), x.shape, y.shape

(pandas.core.frame.DataFrame, pandas.core.series.Series, (1728, 6), (1728,))

In [84]:
# array로 변경하기 전에 dummpy로 변경
x_dumm = pd.get_dummies(x)
x_dumm.shape

(1728, 21)

In [85]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(x_dumm, y)
model.score(x_dumm, y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8912037037037037

In [86]:
x_dumm = np.array(x_dumm)
y = np.array(y)
x_dumm.shape, y.shape

((1728, 21), (1728,))

In [87]:
# OneHotEncoder를 사용하기 위해서는 np.array 형변환 해주어야 한다.
from sklearn.preprocessing import OneHotEncoder
y_2d = y.reshape([-1, 1])
y_hot = OneHotEncoder().fit_transform(y_2d)
y_hot = y_hot.toarray()
y_hot.shape

(1728, 4)

In [88]:
x_dumm = tf.cast(x_dumm, tf.float32)

In [89]:
w = tf.Variable(tf.random_normal(shape=[21, 4]))
b = tf.Variable(tf.random_normal(shape=[4]))
xn = x_dumm / tf.reduce_max(x_dumm)

z = tf.matmul(xn, w)
## z = tf.matmul(xn, w) + b
h = tf.nn.softmax(z)
h = tf.clip_by_value(h, 1e-8, 1- 1e-8) # log(0)이 되지 않도록 안전장치
y_hot = tf.cast(y_hot, tf.float32)
cost = -tf.reduce_mean(y_hot * tf.log(h) + (1-y_hot) * tf.log(1-h))
predict_class = tf.math.argmax(h, axis=1, output_type=tf.int32) # [0, 2]
y_hot = np.array(y_hot)

accuracy = tf.reduce_mean(tf.cast(tf.equal(predict_class, y), tf.float32))
train = tf.train.GradientDescentOptimizer(0.5).minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

TypeError: Input 'y' of 'Equal' Op has type string that does not match type int32 of argument 'x'.

In [90]:
for i in range(10000):
    _, c, a = sess.run([train, cost, accuracy])
    if i%500==0:
        print(i, c, a)

UnimplementedError: Cast string to int32 is not supported
	 [[node Cast_1 (defined at <ipython-input-61-844283289ed2>:12) ]]

Original stack trace for 'Cast_1':
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 539, in run_forever
    self._run_once()
  File "C:\Pro

In [91]:
y

array(['unacc', 'unacc', 'unacc', ..., 'unacc', 'good', 'vgood'],
      dtype=object)